In [42]:
# Import libraries
import os
from os import listdir
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import shapely
import geopandas as gpd
from pyproj import CRS
import utm

In [43]:
# Convert df to gdf
# https://stackoverflow.com/questions/71907567/valueerror-geodataframe-does-not-support-multiple-columns-using-the-geometry-co
# Coordinate system of osm: http://download.geofabrik.de/osm-data-in-gis-formats-free.pdf
# must have a geometry column in csv
def csv_2_gdf(df):
    gdf = gpd.GeoDataFrame(df.loc[:, [c for c in df.columns if c != "geometry"]], 
                           geometry=gpd.GeoSeries.from_wkt(df["geometry"]),
                           crs="epsg:4326",)
    return gdf

### California

In [44]:
file_path = r"D:\Work\Box Sync\TRB 2024\OSM_from_Miguel\OSM Database\California"
os.chdir(file_path)

# Reading and saving the links data as a dict
links = {}
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith("link.csv"):
        # print(file)
        links[file[:-4]] = pd.read_csv(file)  # encoding='cp1252')

print(len(links.keys()))
link_df = pd.concat(links.values()).reset_index(drop=True)
print(link_df.shape) # link_df is a dataframe
print(type(link_df))

143
(10534941, 18)
<class 'pandas.core.frame.DataFrame'>


In [45]:
# read the shapefile for that state
gdf = gpd.read_file(r'D:\Work\Box Sync\TRB 2024\US_places_2020\separateExtract\tl_2020_06_place.zip')
len(gdf), type(gdf)

(1611, geopandas.geodataframe.GeoDataFrame)

In [46]:
import shapely
import geopandas as gpd
from pyproj import CRS
import utm

# Convert df to gdf
# https://stackoverflow.com/questions/71907567/valueerror-geodataframe-does-not-support-multiple-columns-using-the-geometry-co
# Coordinate system of osm: http://download.geofabrik.de/osm-data-in-gis-formats-free.pdf
# must have a geometry column in csv
def csv_2_gdf(df):
    gdf = gpd.GeoDataFrame(df.loc[:, [c for c in df.columns if c != "geometry"]], 
                           geometry=gpd.GeoSeries.from_wkt(df["geometry"]),
                           crs="epsg:4326",)
    return gdf

# assign utm based on the lat long value of a geometry
def utm_crs_from_latlon(lat, lon):
    crs_params = dict(
        proj = 'utm',
        zone = utm.latlon_to_zone_number(lat, lon),
        south = lat < 0
        )
    return CRS.from_dict(crs_params)

# Get projected geometry to create buffer for place polygons
def get_utm_projected(geom_wkt): # geom is shapely geometry file
    geom_shape = shapely.wkt.loads(geom_wkt)
    utm_crs = utm_crs_from_latlon(lat = geom_shape.centroid.y, lon = geom_shape.centroid.x)
    # print(utm_crs)
    line_geo = gpd.GeoSeries([geom_shape], crs='EPSG:4326')
    # taking a buffer of 3.6 meters to include th roads along the periphery
    return line_geo.to_crs(utm_crs).buffer(3.6)

def fill_lanes_with_median_or_one(df, lanes_col='lanes', link_type_col='link_type'):
    # Ensure that lanes_col and link_type_col exist in the dataframe
    if lanes_col not in df.columns or link_type_col not in df.columns:
        raise ValueError(f"Columns {lanes_col} or {link_type_col} not found in the dataframe.")
    
    # Define a function that fills NaN values with the median or 1 if the median is not found
    def fill_na_with_median(group):
        median_value = group[lanes_col].median()
        # If the median value is NaN (all NaN lanes in the group), replace NaN with 1
        if pd.isna(median_value):
            group[lanes_col].fillna(1, inplace=True)
        else:
            group[lanes_col].fillna(median_value, inplace=True)
        return group
    
    # Apply the function to each group defined by the 'link_type' column
    df = df.groupby(link_type_col).apply(fill_na_with_median).reset_index(drop=True)
    
    return df

def custom_agg(series):
    """
    Custom aggregation function:
    - If the number of elements is 4, compute the sum and divide by 2.
    - Otherwise, compute the mean.
    """
    n = len(series)
    if n == 4:
        return series.sum() / 2
    else:
        return series.mean()


In [47]:
# clipping roads at state level to places 
link_df_major = link_df[(link_df['link_type'] < 16)]
state_streets_major = csv_2_gdf(link_df_major) 

# Create a new column with sorted tuples
state_streets_major['sorted_nodes'] = list(zip(state_streets_major['from_node_id'], state_streets_major['to_node_id']))
state_streets_major['sorted_nodes'] = state_streets_major['sorted_nodes'].apply(lambda x: tuple(sorted(x)))
   
clipped_to_places = []

state_places = gdf
# cretae a list of names and geoids
geoids = state_places.GEOID
name_of_places = state_places.NAMELSAD
gdf_wkt = state_places.geometry.to_wkt()  

# Clip link gdf by multipolygons
# Creates a list of clipped files per place in states_places
clipped_streets = [gpd.clip(state_streets_major, get_utm_projected(geom_wkt).to_crs(state_streets_major.crs)) for geom_wkt in gdf_wkt]

print(len(clipped_streets))
print(type(clipped_streets))


1611
<class 'list'>


In [48]:
len(clipped_streets)

1611

In [49]:
clipped_projected_lengths = []
no_roads = []
name_no_roads = []

for i, clipped in enumerate(clipped_streets):
    output_df = pd.DataFrame(clipped)
    # print(len(clipped))    
    print(f'Shape of the roadway clipped by place at index {i} is {clipped.shape}')

    if clipped.shape[0] != 0:
        output_df = pd.DataFrame(clipped)
        utm_length = []
        crs_length = []

        # convert geodataframe geometry to wkt format
        clipped_wkt = clipped.geometry.to_wkt()
        for geom in clipped_wkt:
            # # Convert to accurate UTM: https://gis.stackexchange.com/questions/436938/calculate-length-using-geopandas
            # print(geom)
            # read geometry as a shapely geometry
            line_shape = shapely.wkt.loads(geom)
            # indentify the centroid of the geometry
            utm_crs = utm_crs_from_latlon(lat = line_shape.centroid.y,
                                          lon = line_shape.centroid.x)       

            line_geo = gpd.GeoSeries([line_shape], crs='EPSG:4326')
            crs_length.append(line_geo.to_crs('EPSG:3857').length.values)
            utm_length.append(line_geo.to_crs(utm_crs).length.values)

        if output_df.shape[0] == len(utm_length):
            try:
                output_df['utm_length'] = utm_length
                output_df['crs_length'] = crs_length
            except Exception as e:        
                print(f"!!!====== ERROR ======!!! :\n  {file}")
                print(f"!!!File shapes do not match!!\n")
                continue

        clipped_projected_lengths.append(output_df)
    else:
        print('bleh')
        print('Places and geoids with missing geometry, so no x y values:====')
        print(geoids[i], name_of_places[i])
        no_roads.append(geoids[i])
        name_no_roads.append(name_of_places[i])
        # remove element at index i from the geoid list
        geoids.pop(i)
        name_of_places.pop(i)
        

Shape of the roadway clipped by place at index 0 is (3623, 19)
Shape of the roadway clipped by place at index 1 is (80404, 19)
Shape of the roadway clipped by place at index 2 is (32685, 19)
Shape of the roadway clipped by place at index 3 is (13479, 19)
Shape of the roadway clipped by place at index 4 is (34497, 19)
Shape of the roadway clipped by place at index 5 is (29481, 19)
Shape of the roadway clipped by place at index 6 is (15119, 19)
Shape of the roadway clipped by place at index 7 is (40387, 19)
Shape of the roadway clipped by place at index 8 is (10710, 19)
Shape of the roadway clipped by place at index 9 is (11255, 19)
Shape of the roadway clipped by place at index 10 is (52371, 19)
Shape of the roadway clipped by place at index 11 is (16935, 19)
Shape of the roadway clipped by place at index 12 is (16290, 19)
Shape of the roadway clipped by place at index 13 is (1480, 19)
Shape of the roadway clipped by place at index 14 is (2227, 19)
Shape of the roadway clipped by place 

In [50]:
# ['0600212', '0631092', '0662860'], ['Acton CDP', 'Green Valley CDP', 'Rose Hills CDP']

In [51]:
# clipped_projected_lengths[0]['allowed_uses'].str.contains('walk')

In [52]:
print(len(clipped_projected_lengths))

error_opening = []

motorway_roads = []
trunk_roads = []
primary_roads = []
secondary_roads = []
tertiary_roads = []
unclassified_roads = []
residential_roads = []
service_roads = [] 
track_roads = []
footway_roads = []
cycleway_roads = []
living_street = []

cl_motorway_roads = []
cl_trunk_roads = []
cl_primary_roads = []
cl_secondary_roads = []
cl_tertiary_roads = []
cl_unclassified_roads = []
cl_residential_roads = []
cl_service_roads = [] 
cl_track_roads = []
cl_footway_roads = []
cl_cycleway_roads = []
cl_living_street = []

lane_motorway_roads = []
lane_trunk_roads = []
lane_primary_roads = []
lane_secondary_roads = []
lane_tertiary_roads = []
lane_unclassified_roads = []
lane_residential_roads = []
lane_service_roads = [] 
lane_track_roads = []
lane_footway_roads = []
lane_cycleway_roads = []
lane_living_street = []

n_motorway_roads = []
n_trunk_roads = []
n_primary_roads = []
n_secondary_roads = []
n_tertiary_roads = []
n_unclassified_roads = []
n_residential_roads = []
n_service_roads = [] 
n_track_roads = []
n_footway_roads = []
n_cycleway_roads = []
n_living_street = []

n_cl_motorway_roads = []
n_cl_trunk_roads = []
n_cl_primary_roads = []
n_cl_secondary_roads = []
n_cl_tertiary_roads = []
n_cl_unclassified_roads = []
n_cl_residential_roads = []
n_cl_service_roads = [] 
n_cl_track_roads = []
n_cl_footway_roads = []
n_cl_cycleway_roads = []
n_cl_living_street = []

n_lane_motorway_roads = []
n_lane_trunk_roads = []
n_lane_primary_roads = []
n_lane_secondary_roads = []
n_lane_tertiary_roads = []
n_lane_unclassified_roads = []
n_lane_residential_roads = []
n_lane_service_roads = [] 
n_lane_track_roads = []
n_lane_footway_roads = []
n_lane_cycleway_roads = []
n_lane_living_street = []

for k, clipped_links in enumerate(clipped_projected_lengths):
    try:             
        clipped_links_w_lanes = fill_lanes_with_median_or_one(clipped_links)
        # Filter the data and calculate the sum of road lengths for each road type 
        clipped_links_w_lanes['utm_length'] = clipped_links_w_lanes['utm_length'].astype(str).str.replace("[","").str.replace("]","").astype(float)
        clipped_links_w_lanes['crs_length'] = clipped_links_w_lanes['crs_length'].astype(str).str.replace("[","").str.replace("]","").astype(float)
        clipped_links_w_lanes['lane_m_utm'] = clipped_links_w_lanes['utm_length'] * clipped_links_w_lanes['lanes']
                            
        road_lengths      = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['utm_length'].sum()
        cl_road_lengths   = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby(['link_type_name', 'sorted_nodes', 'utm_length'])['utm_length'].agg(custom_agg).groupby('link_type_name').sum()
        lane_road_lengths = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['lane_m_utm'].sum()
        
        non_auto_rl       = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['utm_length'].sum()
        non_auto_cl_rl    = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby(['link_type_name', 'sorted_nodes', 'utm_length'])['utm_length'].agg(custom_agg).groupby('link_type_name').sum()
        non_auto_lane_rl  = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['lane_m_utm'].sum()

        # walk_road_lengths      = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['utm_length'].sum()
        
        # Append the calculated road lengths to the respective lists
        motorway_roads.append(road_lengths.get('motorway', 0))
        trunk_roads.append(road_lengths.get('trunk', 0))
        primary_roads.append(road_lengths.get('primary', 0))
        secondary_roads.append(road_lengths.get('secondary', 0))
        tertiary_roads.append(road_lengths.get('tertiary', 0))
        unclassified_roads.append(road_lengths.get('unclassified', 0))
        residential_roads.append(road_lengths.get('residential', 0))
        service_roads.append(road_lengths.get('service', 0))
        track_roads.append(road_lengths.get('track', 0))
        footway_roads.append(road_lengths.get('footway', 0))
        cycleway_roads.append(road_lengths.get('cycleway', 0))
        living_street.append(road_lengths.get('living_street', 0))

        cl_motorway_roads.append(cl_road_lengths.get('motorway', 0))
        cl_trunk_roads.append(cl_road_lengths.get('trunk', 0))
        cl_primary_roads.append(cl_road_lengths.get('primary', 0))
        cl_secondary_roads.append(cl_road_lengths.get('secondary', 0))
        cl_tertiary_roads.append(cl_road_lengths.get('tertiary', 0))
        cl_unclassified_roads.append(cl_road_lengths.get('unclassified', 0))
        cl_residential_roads.append(cl_road_lengths.get('residential', 0))
        cl_service_roads.append(cl_road_lengths.get('service', 0))
        cl_track_roads.append(cl_road_lengths.get('track', 0))
        cl_footway_roads.append(cl_road_lengths.get('footway', 0))
        cl_cycleway_roads.append(cl_road_lengths.get('cycleway', 0))
        cl_living_street.append(cl_road_lengths.get('living_street', 0))
        # print("finished upto ", str(k))

        lane_motorway_roads.append(lane_road_lengths.get('motorway', 0))
        lane_trunk_roads.append(lane_road_lengths.get('trunk', 0))
        lane_primary_roads.append(lane_road_lengths.get('primary', 0))
        lane_secondary_roads.append(lane_road_lengths.get('secondary', 0))
        lane_tertiary_roads.append(lane_road_lengths.get('tertiary', 0))
        lane_unclassified_roads.append(lane_road_lengths.get('unclassified', 0))
        lane_residential_roads.append(lane_road_lengths.get('residential', 0))
        lane_service_roads.append(lane_road_lengths.get('service', 0))
        lane_track_roads.append(lane_road_lengths.get('track', 0))
        lane_footway_roads.append(lane_road_lengths.get('footway', 0))
        lane_cycleway_roads.append(lane_road_lengths.get('cycleway', 0))
        lane_living_street.append(lane_road_lengths.get('living_street', 0))

        n_motorway_roads.append(non_auto_rl.get('motorway', 0))
        n_trunk_roads.append(non_auto_rl.get('trunk', 0))
        n_primary_roads.append(non_auto_rl.get('primary', 0))
        n_secondary_roads.append(non_auto_rl.get('secondary', 0))
        n_tertiary_roads.append(non_auto_rl.get('tertiary', 0))
        n_unclassified_roads.append(non_auto_rl.get('unclassified', 0))
        n_residential_roads.append(non_auto_rl.get('residential', 0))
        n_service_roads.append(non_auto_rl.get('service', 0))
        n_track_roads.append(non_auto_rl.get('track', 0))
        n_footway_roads.append(non_auto_rl.get('footway', 0))
        n_cycleway_roads.append(non_auto_rl.get('cycleway', 0))
        n_living_street.append(non_auto_rl.get('living_street', 0))

        n_cl_motorway_roads.append(non_auto_cl_rl.get('motorway', 0))
        n_cl_trunk_roads.append(non_auto_cl_rl.get('trunk', 0))
        n_cl_primary_roads.append(non_auto_cl_rl.get('primary', 0))
        n_cl_secondary_roads.append(non_auto_cl_rl.get('secondary', 0))
        n_cl_tertiary_roads.append(non_auto_cl_rl.get('tertiary', 0))
        n_cl_unclassified_roads.append(non_auto_cl_rl.get('unclassified', 0))
        n_cl_residential_roads.append(non_auto_cl_rl.get('residential', 0))
        n_cl_service_roads.append(non_auto_cl_rl.get('service', 0))
        n_cl_track_roads.append(non_auto_cl_rl.get('track', 0))
        n_cl_footway_roads.append(non_auto_cl_rl.get('footway', 0))
        n_cl_cycleway_roads.append(non_auto_cl_rl.get('cycleway', 0))
        n_cl_living_street.append(non_auto_cl_rl.get('living_street', 0))
        # print("finished upto ", str(k))

        n_lane_motorway_roads.append(non_auto_lane_rl.get('motorway', 0))
        n_lane_trunk_roads.append(non_auto_lane_rl.get('trunk', 0))
        n_lane_primary_roads.append(non_auto_lane_rl.get('primary', 0))
        n_lane_secondary_roads.append(non_auto_lane_rl.get('secondary', 0))
        n_lane_tertiary_roads.append(non_auto_lane_rl.get('tertiary', 0))
        n_lane_unclassified_roads.append(non_auto_lane_rl.get('unclassified', 0))
        n_lane_residential_roads.append(non_auto_lane_rl.get('residential', 0))
        n_lane_service_roads.append(non_auto_lane_rl.get('service', 0))
        n_lane_track_roads.append(non_auto_lane_rl.get('track', 0))
        n_lane_footway_roads.append(non_auto_lane_rl.get('footway', 0))
        n_lane_cycleway_roads.append(non_auto_lane_rl.get('cycleway', 0))
        n_lane_living_street.append(non_auto_lane_rl.get('living_street', 0))
            
    except Exception as e:
        error_opening.append(clipped_links.index)
        print(f"Error opening file clipped_links at position {k} of clipped_projected_lengths: {e}")

1608


c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\num

In [ ]:
list_of_tuples = list(zip(geoids, name_of_places, motorway_roads, trunk_roads, primary_roads, secondary_roads, 
                        tertiary_roads, unclassified_roads, residential_roads, service_roads, track_roads, footway_roads,
                        cycleway_roads, living_street, cl_motorway_roads, cl_trunk_roads, cl_primary_roads,
                        cl_secondary_roads, cl_tertiary_roads, cl_unclassified_roads, cl_residential_roads,
                        cl_service_roads, cl_track_roads, cl_footway_roads, cl_cycleway_roads, cl_living_street,
                        lane_motorway_roads, lane_trunk_roads, lane_primary_roads, lane_secondary_roads, lane_tertiary_roads,
                        lane_unclassified_roads, lane_residential_roads, lane_service_roads, lane_track_roads, lane_footway_roads,
                        lane_cycleway_roads, lane_living_street, n_motorway_roads, n_trunk_roads, n_primary_roads, n_secondary_roads,
                        n_tertiary_roads, n_unclassified_roads, n_residential_roads, n_service_roads, n_track_roads, n_footway_roads, 
                        n_cycleway_roads, n_living_street,n_cl_motorway_roads, n_cl_trunk_roads, n_cl_primary_roads, n_cl_secondary_roads, 
                        n_cl_tertiary_roads, n_cl_unclassified_roads, n_cl_residential_roads, n_cl_service_roads, n_cl_track_roads,
                        n_cl_footway_roads, n_cl_cycleway_roads, n_cl_living_street, n_lane_motorway_roads, n_lane_trunk_roads, 
                        n_lane_primary_roads, n_lane_secondary_roads, n_lane_tertiary_roads, n_lane_unclassified_roads, n_lane_residential_roads,
                        n_lane_service_roads, n_lane_track_roads, n_lane_footway_roads, n_lane_cycleway_roads, n_lane_living_street))

df_roads = pd.DataFrame(list_of_tuples, columns = ['GEOID', 'NAMELSAD','motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                                                    'service', 'track', 'footway', 'cycleway', 'living_street', 
                                                    'cl_motorway', 'cl_trunk', 'cl_primary', 'cl_secondary', 'cl_tertiary', 'cl_unclassified', 'cl_residential',
                                                    'cl_service', 'cl_track', 'cl_footway', 'cl_cycleway', 'cl_living_street',
                                                    'lane_m_motorway', 'lane_m_trunk', 'lane_m_primary', 'lane_m_secondary', 'lane_m_tertiary', 
                                                    'lane_m_unclassified', 'lane_m_residential', 'lane_m_service', 'lane_m_track', 'lane_m_footway',
                                                    'lane_m_cycleway', 'lane_m_living_street',
                                                    'n_motorway', 'n_trunk', 'n_primary', 'n_secondary', 'n_tertiary', 'n_unclassified', 'n_residential',
                                                    'n_service', 'n_track', 'n_footway', 'n_cycleway', 'n_living_street', 
                                                    'n_cl_motorway', 'n_cl_trunk', 'n_cl_primary', 'n_cl_secondary', 'n_cl_tertiary', 'n_cl_unclassified', 'n_cl_residential',
                                                    'n_cl_service', 'n_cl_track', 'n_cl_footway', 'n_cl_cycleway', 'n_cl_living_street',
                                                    'n_lane_m_motorway', 'n_lane_m_trunk', 'n_lane_m_primary', 'n_lane_m_secondary', 'n_lane_m_tertiary', 
                                                    'n_lane_m_unclassified', 'n_lane_m_residential', 'n_lane_m_service', 'n_lane_m_track', 'n_lane_m_footway',
                                                    'n_lane_m_cycleway', 'n_lane_m_living_street'])

print(df_roads.shape)

df_roads.to_csv(r'E:\Scripts\project_QI\data\osm\All states\df_roads_utm_california.csv')

missing_places = {no_roads[i]: name_no_roads[i] for i in range(len(no_roads))}
print(len(missing_places))
missing_places

(1608, 74)
3


{'0600212': 'Acton CDP',
 '0631092': 'Green Valley CDP',
 '0662860': 'Rose Hills CDP'}

In [54]:
# ## SAMPLE TRIAL
# clipped_links = clipped_projected_lengths[1]

# road_lengths = clipped_links.groupby('link_type_name')['utm_length'].sum()
# cl_road_lengths = clipped_links.groupby(['link_type_name', 'sorted_nodes']).agg({'utm_length':'mean'}).groupby('link_type_name').sum()
# # cl_road_lengths = cl_road_lengths.squeeze(axis=1)
# print(type(cl_road_lengths))  
# cl_road_lengths.squeeze()

### Florida

In [55]:
file_path = r"D:\Work\Box Sync\TRB 2024\OSM_from_Miguel\OSM Database\Florida"
os.chdir(file_path)

# Reading and saving the links data as a dict
links = {}
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith("link.csv"):
        # print(file)
        links[file[:-4]] = pd.read_csv(file)  # encoding='cp1252')

print(len(links.keys()))
link_df = pd.concat(links.values()).reset_index(drop=True)
print(link_df.shape) # link_df is a dataframe
print(type(link_df))

67
(8965927, 18)
<class 'pandas.core.frame.DataFrame'>


In [56]:
# read the shapefile for that state
gdf = gpd.read_file(r'D:\Work\Box Sync\TRB 2024\US_places_2020\separateExtract\tl_2020_12_place.zip')
len(gdf), type(gdf)

(955, geopandas.geodataframe.GeoDataFrame)

In [57]:
import shapely
import geopandas as gpd
from pyproj import CRS
import utm

# Convert df to gdf
# https://stackoverflow.com/questions/71907567/valueerror-geodataframe-does-not-support-multiple-columns-using-the-geometry-co
# Coordinate system of osm: http://download.geofabrik.de/osm-data-in-gis-formats-free.pdf
# must have a geometry column in csv
def csv_2_gdf(df):
    gdf = gpd.GeoDataFrame(df.loc[:, [c for c in df.columns if c != "geometry"]], 
                           geometry=gpd.GeoSeries.from_wkt(df["geometry"]),
                           crs="epsg:4326",)
    return gdf

# assign utm based on the lat long value of a geometry
def utm_crs_from_latlon(lat, lon):
    crs_params = dict(
        proj = 'utm',
        zone = utm.latlon_to_zone_number(lat, lon),
        south = lat < 0
        )
    return CRS.from_dict(crs_params)

# Get projected geometry to create buffer for place polygons
def get_utm_projected(geom_wkt): # geom is shapely geometry file
    geom_shape = shapely.wkt.loads(geom_wkt)
    utm_crs = utm_crs_from_latlon(lat = geom_shape.centroid.y, lon = geom_shape.centroid.x)
    # print(utm_crs)
    line_geo = gpd.GeoSeries([geom_shape], crs='EPSG:4326')
    # taking a buffer of 3.6 meters to include th roads along the periphery
    return line_geo.to_crs(utm_crs).buffer(3.6)

def fill_lanes_with_median_or_one(df, lanes_col='lanes', link_type_col='link_type'):
    # Ensure that lanes_col and link_type_col exist in the dataframe
    if lanes_col not in df.columns or link_type_col not in df.columns:
        raise ValueError(f"Columns {lanes_col} or {link_type_col} not found in the dataframe.")
    
    # Define a function that fills NaN values with the median or 1 if the median is not found
    def fill_na_with_median(group):
        median_value = group[lanes_col].median()
        # If the median value is NaN (all NaN lanes in the group), replace NaN with 1
        if pd.isna(median_value):
            group[lanes_col].fillna(1, inplace=True)
        else:
            group[lanes_col].fillna(median_value, inplace=True)
        return group
    
    # Apply the function to each group defined by the 'link_type' column
    df = df.groupby(link_type_col).apply(fill_na_with_median).reset_index(drop=True)
    
    return df

def custom_agg(series):
    """
    Custom aggregation function:
    - If the number of elements is 4, compute the sum and divide by 2.
    - Otherwise, compute the mean.
    """
    n = len(series)
    if n == 4:
        return series.sum() / 2
    else:
        return series.mean()


In [58]:
# clipping roads at state level to places 
link_df_major = link_df[(link_df['link_type'] < 16)]
state_streets_major = csv_2_gdf(link_df_major) 

# Create a new column with sorted tuples
state_streets_major['sorted_nodes'] = list(zip(state_streets_major['from_node_id'], state_streets_major['to_node_id']))
state_streets_major['sorted_nodes'] = state_streets_major['sorted_nodes'].apply(lambda x: tuple(sorted(x)))
   
clipped_to_places = []

state_places = gdf
# cretae a list of names and geoids
geoids = state_places.GEOID
name_of_places = state_places.NAMELSAD
gdf_wkt = state_places.geometry.to_wkt()  

# Clip link gdf by multipolygons
# Creates a list of clipped files per place in states_places
clipped_streets = [gpd.clip(state_streets_major, get_utm_projected(geom_wkt).to_crs(state_streets_major.crs)) for geom_wkt in gdf_wkt]

print(len(clipped_streets))
print(type(clipped_streets))

955
<class 'list'>


In [59]:
clipped_projected_lengths = []
no_roads = []
name_no_roads = []

for i, clipped in enumerate(clipped_streets):
    # print(len(clipped))    
    clipped = clipped[~clipped.is_empty]
    print(f'Shape of the roadway clipped by place at index {i} is {clipped.shape}')
    if clipped.shape[0] != 0:
        output_df = pd.DataFrame(clipped)
        utm_length = []
        crs_length = []

        # convert geodataframe geometry to wkt format
        clipped_wkt = clipped.geometry.to_wkt()
        for geom in clipped_wkt:
            # # Convert to accurate UTM: https://gis.stackexchange.com/questions/436938/calculate-length-using-geopandas
            # print(geom)
            # read geometry as a shapely geometry
            line_shape = shapely.wkt.loads(geom)
            # indentify the centroid of the geometry
            utm_crs = utm_crs_from_latlon(lat = line_shape.centroid.y,
                                          lon = line_shape.centroid.x)       

            line_geo = gpd.GeoSeries([line_shape], crs='EPSG:4326')
            crs_length.append(line_geo.to_crs('EPSG:3857').length.values)
            utm_length.append(line_geo.to_crs(utm_crs).length.values)

        if output_df.shape[0] == len(utm_length):
            try:
                output_df['utm_length'] = utm_length
                output_df['crs_length'] = crs_length
            except Exception as e:        
                print(f"!!!====== ERROR ======!!! :\n  {file}")
                print(f"!!!File shapes do not match!!\n")
                continue

        clipped_projected_lengths.append(output_df)
    else:
        print('bleh')
        print('Places and geoids with missing geometry, so no x y values:====')
        print(geoids[i], name_of_places[i])
        no_roads.append(geoids[i])
        name_no_roads.append(name_of_places[i])
        # remove element at index i from the geoid list
        geoids.pop(i)
        name_of_places.pop(i)
        

Shape of the roadway clipped by place at index 0 is (1250, 19)
Shape of the roadway clipped by place at index 1 is (5550, 19)
Shape of the roadway clipped by place at index 2 is (1258, 19)
Shape of the roadway clipped by place at index 3 is (70117, 19)
Shape of the roadway clipped by place at index 4 is (1290, 19)
Shape of the roadway clipped by place at index 5 is (457, 19)
Shape of the roadway clipped by place at index 6 is (5757, 19)
Shape of the roadway clipped by place at index 7 is (969, 19)
Shape of the roadway clipped by place at index 8 is (10381, 19)
Shape of the roadway clipped by place at index 9 is (10653, 19)
Shape of the roadway clipped by place at index 10 is (4293, 19)
Shape of the roadway clipped by place at index 11 is (12243, 19)
Shape of the roadway clipped by place at index 12 is (2830, 19)
Shape of the roadway clipped by place at index 13 is (1535, 19)
Shape of the roadway clipped by place at index 14 is (1561, 19)
Shape of the roadway clipped by place at index 1

In [60]:
print(len(clipped_projected_lengths))

error_opening = []

motorway_roads = []
trunk_roads = []
primary_roads = []
secondary_roads = []
tertiary_roads = []
unclassified_roads = []
residential_roads = []
service_roads = [] 
track_roads = []
footway_roads = []
cycleway_roads = []
living_street = []

cl_motorway_roads = []
cl_trunk_roads = []
cl_primary_roads = []
cl_secondary_roads = []
cl_tertiary_roads = []
cl_unclassified_roads = []
cl_residential_roads = []
cl_service_roads = [] 
cl_track_roads = []
cl_footway_roads = []
cl_cycleway_roads = []
cl_living_street = []

lane_motorway_roads = []
lane_trunk_roads = []
lane_primary_roads = []
lane_secondary_roads = []
lane_tertiary_roads = []
lane_unclassified_roads = []
lane_residential_roads = []
lane_service_roads = [] 
lane_track_roads = []
lane_footway_roads = []
lane_cycleway_roads = []
lane_living_street = []

n_motorway_roads = []
n_trunk_roads = []
n_primary_roads = []
n_secondary_roads = []
n_tertiary_roads = []
n_unclassified_roads = []
n_residential_roads = []
n_service_roads = [] 
n_track_roads = []
n_footway_roads = []
n_cycleway_roads = []
n_living_street = []

n_cl_motorway_roads = []
n_cl_trunk_roads = []
n_cl_primary_roads = []
n_cl_secondary_roads = []
n_cl_tertiary_roads = []
n_cl_unclassified_roads = []
n_cl_residential_roads = []
n_cl_service_roads = [] 
n_cl_track_roads = []
n_cl_footway_roads = []
n_cl_cycleway_roads = []
n_cl_living_street = []

n_lane_motorway_roads = []
n_lane_trunk_roads = []
n_lane_primary_roads = []
n_lane_secondary_roads = []
n_lane_tertiary_roads = []
n_lane_unclassified_roads = []
n_lane_residential_roads = []
n_lane_service_roads = [] 
n_lane_track_roads = []
n_lane_footway_roads = []
n_lane_cycleway_roads = []
n_lane_living_street = []

for k, clipped_links in enumerate(clipped_projected_lengths):
    try:             
        clipped_links_w_lanes = fill_lanes_with_median_or_one(clipped_links)
        # Filter the data and calculate the sum of road lengths for each road type 
        clipped_links_w_lanes['utm_length'] = clipped_links_w_lanes['utm_length'].astype(str).str.replace("[","").str.replace("]","").astype(float)
        clipped_links_w_lanes['crs_length'] = clipped_links_w_lanes['crs_length'].astype(str).str.replace("[","").str.replace("]","").astype(float)
        clipped_links_w_lanes['lane_m_utm'] = clipped_links_w_lanes['utm_length'] * clipped_links_w_lanes['lanes']
                            
        road_lengths      = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['utm_length'].sum()
        cl_road_lengths   = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby(['link_type_name', 'sorted_nodes', 'utm_length'])['utm_length'].agg(custom_agg).groupby('link_type_name').sum()
        lane_road_lengths = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['lane_m_utm'].sum()
        
        non_auto_rl       = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['utm_length'].sum()
        non_auto_cl_rl    = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby(['link_type_name', 'sorted_nodes', 'utm_length'])['utm_length'].agg(custom_agg).groupby('link_type_name').sum()
        non_auto_lane_rl  = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['lane_m_utm'].sum()
        
        # Append the calculated road lengths to the respective lists
        motorway_roads.append(road_lengths.get('motorway', 0))
        trunk_roads.append(road_lengths.get('trunk', 0))
        primary_roads.append(road_lengths.get('primary', 0))
        secondary_roads.append(road_lengths.get('secondary', 0))
        tertiary_roads.append(road_lengths.get('tertiary', 0))
        unclassified_roads.append(road_lengths.get('unclassified', 0))
        residential_roads.append(road_lengths.get('residential', 0))
        service_roads.append(road_lengths.get('service', 0))
        track_roads.append(road_lengths.get('track', 0))
        footway_roads.append(road_lengths.get('footway', 0))
        cycleway_roads.append(road_lengths.get('cycleway', 0))
        living_street.append(road_lengths.get('living_street', 0))

        cl_motorway_roads.append(cl_road_lengths.get('motorway', 0))
        cl_trunk_roads.append(cl_road_lengths.get('trunk', 0))
        cl_primary_roads.append(cl_road_lengths.get('primary', 0))
        cl_secondary_roads.append(cl_road_lengths.get('secondary', 0))
        cl_tertiary_roads.append(cl_road_lengths.get('tertiary', 0))
        cl_unclassified_roads.append(cl_road_lengths.get('unclassified', 0))
        cl_residential_roads.append(cl_road_lengths.get('residential', 0))
        cl_service_roads.append(cl_road_lengths.get('service', 0))
        cl_track_roads.append(cl_road_lengths.get('track', 0))
        cl_footway_roads.append(cl_road_lengths.get('footway', 0))
        cl_cycleway_roads.append(cl_road_lengths.get('cycleway', 0))
        cl_living_street.append(cl_road_lengths.get('living_street', 0))
        # print("finished upto ", str(k))

        lane_motorway_roads.append(lane_road_lengths.get('motorway', 0))
        lane_trunk_roads.append(lane_road_lengths.get('trunk', 0))
        lane_primary_roads.append(lane_road_lengths.get('primary', 0))
        lane_secondary_roads.append(lane_road_lengths.get('secondary', 0))
        lane_tertiary_roads.append(lane_road_lengths.get('tertiary', 0))
        lane_unclassified_roads.append(lane_road_lengths.get('unclassified', 0))
        lane_residential_roads.append(lane_road_lengths.get('residential', 0))
        lane_service_roads.append(lane_road_lengths.get('service', 0))
        lane_track_roads.append(lane_road_lengths.get('track', 0))
        lane_footway_roads.append(lane_road_lengths.get('footway', 0))
        lane_cycleway_roads.append(lane_road_lengths.get('cycleway', 0))
        lane_living_street.append(lane_road_lengths.get('living_street', 0))

        n_motorway_roads.append(non_auto_rl.get('motorway', 0))
        n_trunk_roads.append(non_auto_rl.get('trunk', 0))
        n_primary_roads.append(non_auto_rl.get('primary', 0))
        n_secondary_roads.append(non_auto_rl.get('secondary', 0))
        n_tertiary_roads.append(non_auto_rl.get('tertiary', 0))
        n_unclassified_roads.append(non_auto_rl.get('unclassified', 0))
        n_residential_roads.append(non_auto_rl.get('residential', 0))
        n_service_roads.append(non_auto_rl.get('service', 0))
        n_track_roads.append(non_auto_rl.get('track', 0))
        n_footway_roads.append(non_auto_rl.get('footway', 0))
        n_cycleway_roads.append(non_auto_rl.get('cycleway', 0))
        n_living_street.append(non_auto_rl.get('living_street', 0))

        n_cl_motorway_roads.append(non_auto_cl_rl.get('motorway', 0))
        n_cl_trunk_roads.append(non_auto_cl_rl.get('trunk', 0))
        n_cl_primary_roads.append(non_auto_cl_rl.get('primary', 0))
        n_cl_secondary_roads.append(non_auto_cl_rl.get('secondary', 0))
        n_cl_tertiary_roads.append(non_auto_cl_rl.get('tertiary', 0))
        n_cl_unclassified_roads.append(non_auto_cl_rl.get('unclassified', 0))
        n_cl_residential_roads.append(non_auto_cl_rl.get('residential', 0))
        n_cl_service_roads.append(non_auto_cl_rl.get('service', 0))
        n_cl_track_roads.append(non_auto_cl_rl.get('track', 0))
        n_cl_footway_roads.append(non_auto_cl_rl.get('footway', 0))
        n_cl_cycleway_roads.append(non_auto_cl_rl.get('cycleway', 0))
        n_cl_living_street.append(non_auto_cl_rl.get('living_street', 0))
        # print("finished upto ", str(k))

        n_lane_motorway_roads.append(non_auto_lane_rl.get('motorway', 0))
        n_lane_trunk_roads.append(non_auto_lane_rl.get('trunk', 0))
        n_lane_primary_roads.append(non_auto_lane_rl.get('primary', 0))
        n_lane_secondary_roads.append(non_auto_lane_rl.get('secondary', 0))
        n_lane_tertiary_roads.append(non_auto_lane_rl.get('tertiary', 0))
        n_lane_unclassified_roads.append(non_auto_lane_rl.get('unclassified', 0))
        n_lane_residential_roads.append(non_auto_lane_rl.get('residential', 0))
        n_lane_service_roads.append(non_auto_lane_rl.get('service', 0))
        n_lane_track_roads.append(non_auto_lane_rl.get('track', 0))
        n_lane_footway_roads.append(non_auto_lane_rl.get('footway', 0))
        n_lane_cycleway_roads.append(non_auto_lane_rl.get('cycleway', 0))
        n_lane_living_street.append(non_auto_lane_rl.get('living_street', 0))
            
    except Exception as e:
        error_opening.append(clipped_links.index)
        print(f"Error opening file clipped_links at position {k} of clipped_projected_lengths: {e}")

955


c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\num

In [ ]:
list_of_tuples = list(zip(geoids, name_of_places, motorway_roads, trunk_roads, primary_roads, secondary_roads, 
                        tertiary_roads, unclassified_roads, residential_roads, service_roads, track_roads, footway_roads,
                        cycleway_roads, living_street, cl_motorway_roads, cl_trunk_roads, cl_primary_roads,
                        cl_secondary_roads, cl_tertiary_roads, cl_unclassified_roads, cl_residential_roads,
                        cl_service_roads, cl_track_roads, cl_footway_roads, cl_cycleway_roads, cl_living_street,
                        lane_motorway_roads, lane_trunk_roads, lane_primary_roads, lane_secondary_roads, lane_tertiary_roads,
                        lane_unclassified_roads, lane_residential_roads, lane_service_roads, lane_track_roads, lane_footway_roads,
                        lane_cycleway_roads, lane_living_street, n_motorway_roads, n_trunk_roads, n_primary_roads, n_secondary_roads,
                        n_tertiary_roads, n_unclassified_roads, n_residential_roads, n_service_roads, n_track_roads, n_footway_roads, 
                        n_cycleway_roads, n_living_street,n_cl_motorway_roads, n_cl_trunk_roads, n_cl_primary_roads, n_cl_secondary_roads, 
                        n_cl_tertiary_roads, n_cl_unclassified_roads, n_cl_residential_roads, n_cl_service_roads, n_cl_track_roads,
                        n_cl_footway_roads, n_cl_cycleway_roads, n_cl_living_street, n_lane_motorway_roads, n_lane_trunk_roads, 
                        n_lane_primary_roads, n_lane_secondary_roads, n_lane_tertiary_roads, n_lane_unclassified_roads, n_lane_residential_roads,
                        n_lane_service_roads, n_lane_track_roads, n_lane_footway_roads, n_lane_cycleway_roads, n_lane_living_street))

df_roads = pd.DataFrame(list_of_tuples, columns = ['GEOID', 'NAMELSAD','motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                                                    'service', 'track', 'footway', 'cycleway', 'living_street', 
                                                    'cl_motorway', 'cl_trunk', 'cl_primary', 'cl_secondary', 'cl_tertiary', 'cl_unclassified', 'cl_residential',
                                                    'cl_service', 'cl_track', 'cl_footway', 'cl_cycleway', 'cl_living_street',
                                                    'lane_m_motorway', 'lane_m_trunk', 'lane_m_primary', 'lane_m_secondary', 'lane_m_tertiary', 
                                                    'lane_m_unclassified', 'lane_m_residential', 'lane_m_service', 'lane_m_track', 'lane_m_footway',
                                                    'lane_m_cycleway', 'lane_m_living_street',
                                                    'n_motorway', 'n_trunk', 'n_primary', 'n_secondary', 'n_tertiary', 'n_unclassified', 'n_residential',
                                                    'n_service', 'n_track', 'n_footway', 'n_cycleway', 'n_living_street', 
                                                    'n_cl_motorway', 'n_cl_trunk', 'n_cl_primary', 'n_cl_secondary', 'n_cl_tertiary', 'n_cl_unclassified', 'n_cl_residential',
                                                    'n_cl_service', 'n_cl_track', 'n_cl_footway', 'n_cl_cycleway', 'n_cl_living_street',
                                                    'n_lane_m_motorway', 'n_lane_m_trunk', 'n_lane_m_primary', 'n_lane_m_secondary', 'n_lane_m_tertiary', 
                                                    'n_lane_m_unclassified', 'n_lane_m_residential', 'n_lane_m_service', 'n_lane_m_track', 'n_lane_m_footway',
                                                    'n_lane_m_cycleway', 'n_lane_m_living_street'])

print(df_roads.shape)
df_roads.to_csv(r'E:\Scripts\project_QI\data\osm\All states\df_roads_utm_florida.csv')

missing_places = {no_roads[i]: name_no_roads[i] for i in range(len(no_roads))}
print(len(missing_places))

(955, 74)
0


In [62]:
missing_places

{}

### Texas

In [63]:
file_path = r"D:\Work\Box Sync\TRB 2024\OSM_from_Miguel\OSM Database\Texas"
os.chdir(file_path)

# Reading and saving the links data as a dict
links = {}
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith("link.csv"):
        # print(file)
        links[file[:-4]] = pd.read_csv(file)  # encoding='cp1252')

print(len(links.keys()))
link_df = pd.concat(links.values()).reset_index(drop=True)
print(link_df.shape) # link_df is a dataframe
print(type(link_df))

254
(12740826, 18)
<class 'pandas.core.frame.DataFrame'>


In [64]:
# read the shapefile for that state
gdf = gpd.read_file(r'D:\Work\Box Sync\TRB 2024\US_places_2020\separateExtract\tl_2020_48_place.zip')
len(gdf), type(gdf)

(1860, geopandas.geodataframe.GeoDataFrame)

In [65]:
# clipping roads at state level to places 
link_df_major = link_df[(link_df['link_type'] < 16)]
state_streets_major = csv_2_gdf(link_df_major) 

# Create a new column with sorted tuples
state_streets_major['sorted_nodes'] = list(zip(state_streets_major['from_node_id'], state_streets_major['to_node_id']))
state_streets_major['sorted_nodes'] = state_streets_major['sorted_nodes'].apply(lambda x: tuple(sorted(x)))
   
clipped_to_places = []

state_places = gdf
# cretae a list of names and geoids
geoids = state_places.GEOID
name_of_places = state_places.NAMELSAD
gdf_wkt = state_places.geometry.to_wkt()  

# Clip link gdf by multipolygons
# Creates a list of clipped files per place in states_places
clipped_streets = [gpd.clip(state_streets_major, get_utm_projected(geom_wkt).to_crs(state_streets_major.crs)) for geom_wkt in gdf_wkt]

print(len(clipped_streets))
print(type(clipped_streets))


1860
<class 'list'>


In [66]:
clipped_projected_lengths = []
no_roads = []
name_no_roads = []

for i, clipped in enumerate(clipped_streets):
    # print(len(clipped))    
    clipped = clipped[~clipped.is_empty]
    print(f'Shape of the roadway clipped by place at index {i} is {clipped.shape}')
    if clipped.shape[0] != 0:
        output_df = pd.DataFrame(clipped)
        utm_length = []
        crs_length = []

        # convert geodataframe geometry to wkt format
        clipped_wkt = clipped.geometry.to_wkt()
        for geom in clipped_wkt:
            # # Convert to accurate UTM: https://gis.stackexchange.com/questions/436938/calculate-length-using-geopandas
            # print(geom)
            # read geometry as a shapely geometry
            line_shape = shapely.wkt.loads(geom)
            # indentify the centroid of the geometry
            utm_crs = utm_crs_from_latlon(lat = line_shape.centroid.y,
                                          lon = line_shape.centroid.x)       

            line_geo = gpd.GeoSeries([line_shape], crs='EPSG:4326')
            crs_length.append(line_geo.to_crs('EPSG:3857').length.values)
            utm_length.append(line_geo.to_crs(utm_crs).length.values)

        if output_df.shape[0] == len(utm_length):
            try:
                output_df['utm_length'] = utm_length
                output_df['crs_length'] = crs_length
            except Exception as e:        
                print(f"!!!====== ERROR ======!!! :\n  {file}")
                print(f"!!!File shapes do not match!!\n")
                continue

        clipped_projected_lengths.append(output_df)
    else:
        print('bleh')
        print('Places and geoids with missing geometry, so no x y values:====')
        print(geoids[i], name_of_places[i])
        no_roads.append(geoids[i])
        name_no_roads.append(name_of_places[i])
        # remove element at index i from the geoid list
        geoids.pop(i)
        name_of_places.pop(i)
        

Shape of the roadway clipped by place at index 0 is (5235, 19)
Shape of the roadway clipped by place at index 1 is (456, 19)
Shape of the roadway clipped by place at index 2 is (1869, 19)
Shape of the roadway clipped by place at index 3 is (244, 19)
Shape of the roadway clipped by place at index 4 is (286, 19)
Shape of the roadway clipped by place at index 5 is (160, 19)
Shape of the roadway clipped by place at index 6 is (1205, 19)
Shape of the roadway clipped by place at index 7 is (12680, 19)
Shape of the roadway clipped by place at index 8 is (284, 19)
Shape of the roadway clipped by place at index 9 is (682, 19)
Shape of the roadway clipped by place at index 10 is (839, 19)
Shape of the roadway clipped by place at index 11 is (254, 19)
Shape of the roadway clipped by place at index 12 is (191, 19)
Shape of the roadway clipped by place at index 13 is (1073, 19)
Shape of the roadway clipped by place at index 14 is (2541, 19)
Shape of the roadway clipped by place at index 15 is (847,

In [67]:
print(len(clipped_projected_lengths))

error_opening = []

motorway_roads = []
trunk_roads = []
primary_roads = []
secondary_roads = []
tertiary_roads = []
unclassified_roads = []
residential_roads = []
service_roads = [] 
track_roads = []
footway_roads = []
cycleway_roads = []
living_street = []

cl_motorway_roads = []
cl_trunk_roads = []
cl_primary_roads = []
cl_secondary_roads = []
cl_tertiary_roads = []
cl_unclassified_roads = []
cl_residential_roads = []
cl_service_roads = [] 
cl_track_roads = []
cl_footway_roads = []
cl_cycleway_roads = []
cl_living_street = []

lane_motorway_roads = []
lane_trunk_roads = []
lane_primary_roads = []
lane_secondary_roads = []
lane_tertiary_roads = []
lane_unclassified_roads = []
lane_residential_roads = []
lane_service_roads = [] 
lane_track_roads = []
lane_footway_roads = []
lane_cycleway_roads = []
lane_living_street = []

n_motorway_roads = []
n_trunk_roads = []
n_primary_roads = []
n_secondary_roads = []
n_tertiary_roads = []
n_unclassified_roads = []
n_residential_roads = []
n_service_roads = [] 
n_track_roads = []
n_footway_roads = []
n_cycleway_roads = []
n_living_street = []

n_cl_motorway_roads = []
n_cl_trunk_roads = []
n_cl_primary_roads = []
n_cl_secondary_roads = []
n_cl_tertiary_roads = []
n_cl_unclassified_roads = []
n_cl_residential_roads = []
n_cl_service_roads = [] 
n_cl_track_roads = []
n_cl_footway_roads = []
n_cl_cycleway_roads = []
n_cl_living_street = []

n_lane_motorway_roads = []
n_lane_trunk_roads = []
n_lane_primary_roads = []
n_lane_secondary_roads = []
n_lane_tertiary_roads = []
n_lane_unclassified_roads = []
n_lane_residential_roads = []
n_lane_service_roads = [] 
n_lane_track_roads = []
n_lane_footway_roads = []
n_lane_cycleway_roads = []
n_lane_living_street = []

for k, clipped_links in enumerate(clipped_projected_lengths):
    try:             
        clipped_links_w_lanes = fill_lanes_with_median_or_one(clipped_links)
        # Filter the data and calculate the sum of road lengths for each road type 
        clipped_links_w_lanes['utm_length'] = clipped_links_w_lanes['utm_length'].astype(str).str.replace("[","").str.replace("]","").astype(float)
        clipped_links_w_lanes['crs_length'] = clipped_links_w_lanes['crs_length'].astype(str).str.replace("[","").str.replace("]","").astype(float)
        clipped_links_w_lanes['lane_m_utm'] = clipped_links_w_lanes['utm_length'] * clipped_links_w_lanes['lanes']
                            
        road_lengths      = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['utm_length'].sum()
        cl_road_lengths   = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby(['link_type_name', 'sorted_nodes', 'utm_length'])['utm_length'].agg(custom_agg).groupby('link_type_name').sum()
        lane_road_lengths = clipped_links_w_lanes[clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['lane_m_utm'].sum()
        
        non_auto_rl       = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['utm_length'].sum()
        non_auto_cl_rl    = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby(['link_type_name', 'sorted_nodes', 'utm_length'])['utm_length'].agg(custom_agg).groupby('link_type_name').sum()
        non_auto_lane_rl  = clipped_links_w_lanes[~clipped_links_w_lanes['allowed_uses'].str.contains('auto')].groupby('link_type_name')['lane_m_utm'].sum()
        
        # Append the calculated road lengths to the respective lists
        motorway_roads.append(road_lengths.get('motorway', 0))
        trunk_roads.append(road_lengths.get('trunk', 0))
        primary_roads.append(road_lengths.get('primary', 0))
        secondary_roads.append(road_lengths.get('secondary', 0))
        tertiary_roads.append(road_lengths.get('tertiary', 0))
        unclassified_roads.append(road_lengths.get('unclassified', 0))
        residential_roads.append(road_lengths.get('residential', 0))
        service_roads.append(road_lengths.get('service', 0))
        track_roads.append(road_lengths.get('track', 0))
        footway_roads.append(road_lengths.get('footway', 0))
        cycleway_roads.append(road_lengths.get('cycleway', 0))
        living_street.append(road_lengths.get('living_street', 0))

        cl_motorway_roads.append(cl_road_lengths.get('motorway', 0))
        cl_trunk_roads.append(cl_road_lengths.get('trunk', 0))
        cl_primary_roads.append(cl_road_lengths.get('primary', 0))
        cl_secondary_roads.append(cl_road_lengths.get('secondary', 0))
        cl_tertiary_roads.append(cl_road_lengths.get('tertiary', 0))
        cl_unclassified_roads.append(cl_road_lengths.get('unclassified', 0))
        cl_residential_roads.append(cl_road_lengths.get('residential', 0))
        cl_service_roads.append(cl_road_lengths.get('service', 0))
        cl_track_roads.append(cl_road_lengths.get('track', 0))
        cl_footway_roads.append(cl_road_lengths.get('footway', 0))
        cl_cycleway_roads.append(cl_road_lengths.get('cycleway', 0))
        cl_living_street.append(cl_road_lengths.get('living_street', 0))
        # print("finished upto ", str(k))

        lane_motorway_roads.append(lane_road_lengths.get('motorway', 0))
        lane_trunk_roads.append(lane_road_lengths.get('trunk', 0))
        lane_primary_roads.append(lane_road_lengths.get('primary', 0))
        lane_secondary_roads.append(lane_road_lengths.get('secondary', 0))
        lane_tertiary_roads.append(lane_road_lengths.get('tertiary', 0))
        lane_unclassified_roads.append(lane_road_lengths.get('unclassified', 0))
        lane_residential_roads.append(lane_road_lengths.get('residential', 0))
        lane_service_roads.append(lane_road_lengths.get('service', 0))
        lane_track_roads.append(lane_road_lengths.get('track', 0))
        lane_footway_roads.append(lane_road_lengths.get('footway', 0))
        lane_cycleway_roads.append(lane_road_lengths.get('cycleway', 0))
        lane_living_street.append(lane_road_lengths.get('living_street', 0))

        n_motorway_roads.append(non_auto_rl.get('motorway', 0))
        n_trunk_roads.append(non_auto_rl.get('trunk', 0))
        n_primary_roads.append(non_auto_rl.get('primary', 0))
        n_secondary_roads.append(non_auto_rl.get('secondary', 0))
        n_tertiary_roads.append(non_auto_rl.get('tertiary', 0))
        n_unclassified_roads.append(non_auto_rl.get('unclassified', 0))
        n_residential_roads.append(non_auto_rl.get('residential', 0))
        n_service_roads.append(non_auto_rl.get('service', 0))
        n_track_roads.append(non_auto_rl.get('track', 0))
        n_footway_roads.append(non_auto_rl.get('footway', 0))
        n_cycleway_roads.append(non_auto_rl.get('cycleway', 0))
        n_living_street.append(non_auto_rl.get('living_street', 0))

        n_cl_motorway_roads.append(non_auto_cl_rl.get('motorway', 0))
        n_cl_trunk_roads.append(non_auto_cl_rl.get('trunk', 0))
        n_cl_primary_roads.append(non_auto_cl_rl.get('primary', 0))
        n_cl_secondary_roads.append(non_auto_cl_rl.get('secondary', 0))
        n_cl_tertiary_roads.append(non_auto_cl_rl.get('tertiary', 0))
        n_cl_unclassified_roads.append(non_auto_cl_rl.get('unclassified', 0))
        n_cl_residential_roads.append(non_auto_cl_rl.get('residential', 0))
        n_cl_service_roads.append(non_auto_cl_rl.get('service', 0))
        n_cl_track_roads.append(non_auto_cl_rl.get('track', 0))
        n_cl_footway_roads.append(non_auto_cl_rl.get('footway', 0))
        n_cl_cycleway_roads.append(non_auto_cl_rl.get('cycleway', 0))
        n_cl_living_street.append(non_auto_cl_rl.get('living_street', 0))
        # print("finished upto ", str(k))

        n_lane_motorway_roads.append(non_auto_lane_rl.get('motorway', 0))
        n_lane_trunk_roads.append(non_auto_lane_rl.get('trunk', 0))
        n_lane_primary_roads.append(non_auto_lane_rl.get('primary', 0))
        n_lane_secondary_roads.append(non_auto_lane_rl.get('secondary', 0))
        n_lane_tertiary_roads.append(non_auto_lane_rl.get('tertiary', 0))
        n_lane_unclassified_roads.append(non_auto_lane_rl.get('unclassified', 0))
        n_lane_residential_roads.append(non_auto_lane_rl.get('residential', 0))
        n_lane_service_roads.append(non_auto_lane_rl.get('service', 0))
        n_lane_track_roads.append(non_auto_lane_rl.get('track', 0))
        n_lane_footway_roads.append(non_auto_lane_rl.get('footway', 0))
        n_lane_cycleway_roads.append(non_auto_lane_rl.get('cycleway', 0))
        n_lane_living_street.append(non_auto_lane_rl.get('living_street', 0))
            
    except Exception as e:
        error_opening.append(clipped_links.index)
        print(f"Error opening file clipped_links at position {k} of clipped_projected_lengths: {e}")

1860


c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\uttar\miniforge3\envs\analyzeInfra\Lib\site-packages\num

In [ ]:
list_of_tuples = list(zip(geoids, name_of_places, motorway_roads, trunk_roads, primary_roads, secondary_roads, 
                        tertiary_roads, unclassified_roads, residential_roads, service_roads, track_roads, footway_roads,
                        cycleway_roads, living_street, cl_motorway_roads, cl_trunk_roads, cl_primary_roads,
                        cl_secondary_roads, cl_tertiary_roads, cl_unclassified_roads, cl_residential_roads,
                        cl_service_roads, cl_track_roads, cl_footway_roads, cl_cycleway_roads, cl_living_street,
                        lane_motorway_roads, lane_trunk_roads, lane_primary_roads, lane_secondary_roads, lane_tertiary_roads,
                        lane_unclassified_roads, lane_residential_roads, lane_service_roads, lane_track_roads, lane_footway_roads,
                        lane_cycleway_roads, lane_living_street, n_motorway_roads, n_trunk_roads, n_primary_roads, n_secondary_roads,
                        n_tertiary_roads, n_unclassified_roads, n_residential_roads, n_service_roads, n_track_roads, n_footway_roads, 
                        n_cycleway_roads, n_living_street,n_cl_motorway_roads, n_cl_trunk_roads, n_cl_primary_roads, n_cl_secondary_roads, 
                        n_cl_tertiary_roads, n_cl_unclassified_roads, n_cl_residential_roads, n_cl_service_roads, n_cl_track_roads,
                        n_cl_footway_roads, n_cl_cycleway_roads, n_cl_living_street, n_lane_motorway_roads, n_lane_trunk_roads, 
                        n_lane_primary_roads, n_lane_secondary_roads, n_lane_tertiary_roads, n_lane_unclassified_roads, n_lane_residential_roads,
                        n_lane_service_roads, n_lane_track_roads, n_lane_footway_roads, n_lane_cycleway_roads, n_lane_living_street))

df_roads = pd.DataFrame(list_of_tuples, columns = ['GEOID', 'NAMELSAD','motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                                                    'service', 'track', 'footway', 'cycleway', 'living_street', 
                                                    'cl_motorway', 'cl_trunk', 'cl_primary', 'cl_secondary', 'cl_tertiary', 'cl_unclassified', 'cl_residential',
                                                    'cl_service', 'cl_track', 'cl_footway', 'cl_cycleway', 'cl_living_street',
                                                    'lane_m_motorway', 'lane_m_trunk', 'lane_m_primary', 'lane_m_secondary', 'lane_m_tertiary', 
                                                    'lane_m_unclassified', 'lane_m_residential', 'lane_m_service', 'lane_m_track', 'lane_m_footway',
                                                    'lane_m_cycleway', 'lane_m_living_street',
                                                    'n_motorway', 'n_trunk', 'n_primary', 'n_secondary', 'n_tertiary', 'n_unclassified', 'n_residential',
                                                    'n_service', 'n_track', 'n_footway', 'n_cycleway', 'n_living_street', 
                                                    'n_cl_motorway', 'n_cl_trunk', 'n_cl_primary', 'n_cl_secondary', 'n_cl_tertiary', 'n_cl_unclassified', 'n_cl_residential',
                                                    'n_cl_service', 'n_cl_track', 'n_cl_footway', 'n_cl_cycleway', 'n_cl_living_street',
                                                    'n_lane_m_motorway', 'n_lane_m_trunk', 'n_lane_m_primary', 'n_lane_m_secondary', 'n_lane_m_tertiary', 
                                                    'n_lane_m_unclassified', 'n_lane_m_residential', 'n_lane_m_service', 'n_lane_m_track', 'n_lane_m_footway',
                                                    'n_lane_m_cycleway', 'n_lane_m_living_street'])

print(df_roads.shape)
df_roads.to_csv(r'E:\Scripts\project_QI\data\osm\All states\df_roads_utm_texas.csv')

missing_places = {no_roads[i]: name_no_roads[i] for i in range(len(no_roads))}
print(len(missing_places))

(1860, 74)
0


In [69]:
missing_places

{}

In [5]:
# # assign utm based on the lat long value of a geometry
# def utm_crs_from_latlon(lat, lon):
#     crs_params = dict(
#         proj = 'utm',
#         zone = utm.latlon_to_zone_number(lat, lon),
#         south = lat < 0
#         )
#     return CRS.from_dict(crs_params)

# # Get projected geometry to create buffer for place polygons
# def get_utm_projected(geom_wkt): # geom is shapely geometry file
#     geom_shape = shapely.wkt.loads(geom_wkt)
#     utm_crs = utm_crs_from_latlon(lat = geom_shape.centroid.y, lon = geom_shape.centroid.x)
#     # print(utm_crs)
#     line_geo = gpd.GeoSeries([geom_shape], crs='EPSG:4326')
#     # taking a buffer of 3.6 meters to include th roads along the periphery
#     return line_geo.to_crs(utm_crs).buffer(3.6)

In [ ]:
# TRIAL & CHECK

In [6]:
gdf = gdf[gdf['NAMELSAD'].str.startswith('Houston')]

In [ ]:
link_df

In [ ]:
link_df[(link_df['link_type'] < 16)].groupby(['link_type', 'link_type_name'])[['length', 'lanes']].mean()

In [ ]:
link_df[(link_df['link_type_name'] =='service')]['allowed_uses'].unique()

In [279]:
import pandas as pd

def fill_lanes_with_median(df, lanes_col='lanes', link_type_col='link_type'):
    # Ensure that lanes_col and link_type_col exist in the dataframe
    if lanes_col not in df.columns or link_type_col not in df.columns:
        raise ValueError(f"Columns {lanes_col} or {link_type_col} not found in the dataframe.")
    
    # Define a function that fills NaN values with the median of the corresponding link_type group
    def fill_na_with_median(group):
        median_value = group[lanes_col].median()
        group[lanes_col].fillna(median_value, inplace=True)
        return group
    
    # Apply the function to each group defined by the 'link_type' column
    df = df.groupby(link_type_col).apply(fill_na_with_median)
    
    return df

In [283]:
df_roads = link_df[(link_df['link_type'] < 16)]
df_filled = fill_lanes_with_median(df_roads)

In [ ]:
state_streets_major[(state_streets_major['link_type'] == 11) & (state_streets_major['allowed_uses'].str.contains('auto'))].plot()

In [ ]:
np.sum(df_filled['length'] * df_filled['lanes'])

In [ ]:
np.sum(df_filled['length'] * df_filled['lanes'])

In [ ]:
df_roads[df_roads['link_type'] == 1]['length'].sum()* 13.6 + df_roads[df_roads['link_type'] == 2]['length'].sum()* 9.6 + df_roads[df_roads['link_type'] == 3]['length'].sum()* 6 + df_roads[df_roads['link_type'] == 4]['length'].sum()* 5.3 +\
df_roads[df_roads['link_type'] == 5]['length'].sum()* 4.9 + df_roads[df_roads['link_type'] == 6]['length'].sum()* 4.5 + df_roads[df_roads['link_type'] == 7]['length'].sum()* 4.5 + df_roads[df_roads['link_type'] == 8]['length'].sum()* 4.5 + \
df_roads[df_roads['link_type'] == 9]['length'].sum()* 4.5 + df_roads[df_roads['link_type'] == 10]['length'].sum()* 4.5 + df_roads[df_roads['link_type'] == 11]['length'].sum()* 4.5 + df_roads[df_roads['link_type'] == 15]['length'].sum()* 4.5 

In [ ]:
11129001127.801992/3.7

In [304]:
# clipping roads at state level to places 
link_df_major = link_df[(link_df['link_type'] < 16)]
state_streets_major = csv_2_gdf(link_df_major) 

# Create a new column with sorted tuples
state_streets_major['sorted_nodes'] = list(zip(state_streets_major['from_node_id'], state_streets_major['to_node_id']))
state_streets_major['sorted_nodes'] = state_streets_major['sorted_nodes'].apply(lambda x: tuple(sorted(x)))

In [57]:
from shapely.ops import unary_union
state_places = gdf
state_places = state_places.to_crs(state_streets_major.crs)
clipped_geom = unary_union(state_places['geometry'].to_list())


In [38]:
clipped_df = gpd.clip(state_streets_major, clipped_geom)

In [ ]:
clipped_df.groupby(['sorted_nodes','from_node_id','length']).size()

In [ ]:
clipped_df.groupby(['link_type_name', 'link_type'])['length'].sum()

In [311]:
df_check = clipped_df.groupby(['link_type_name', 'from_node_id','link_type', 'length']).size().reset_index()

In [ ]:
df_check[df_check[0]>3]

In [ ]:
clipped_df[clipped_df['link_type'] > 4].plot(column='link_type', legend =True) #.groupby(['link_type_name', 'sorted_nodes', 'length']).agg({'length':'mean'}).groupby('link_type_name').sum()

In [ ]:
clipped_df.groupby(['link_type_name', 'sorted_nodes', 'length']).agg({'length':'mean'}).groupby('link_type_name').sum()

In [ ]:
clipped_df[clipped_df['link_type_name'] == 'unclassified'].plot(column ='lanes', legend = True) #.plot() #.str.startswith('Forest')]

In [ ]:
df_place = clipped_streets[0]
df_place.groupby(['link_type_name', 'sorted_nodes', 'length']).agg({'length':'mean'}).groupby('link_type_name').sum()

In [ ]:
df_place[df_place['link_type']<5].plot(column='link_type_name', legend =True) 

In [ ]:
df_place[df_place['link_type']>5].plot(column='link_type_name', legend =True) 

In [ ]:
df_place[df_place['link_type']> 4].plot(column='link_type_name', legend =True) 

In [ ]:
df_place[['name', 'link_id', 'osm_way_id', 'from_node_id', 'to_node_id',
       'dir_flag', 'length', 'lanes', 'free_speed', 'capacity',
       'link_type_name', 'link_type', 'allowed_uses', 'from_biway', 'is_link',
       'VDF_fftt1', 'VDF_cap1', 'sorted_nodes']].to_csv(r'E:\Scripts\project_QI\outputfiles\csvs\df_houston.csv')

In [ ]:
df_place.groupby(['link_type','link_type_name'])['lanes'].describe()

In [348]:
df_size = df_place.groupby(['link_type_name', 'from_node_id', 'to_node_id','sorted_nodes', 'length']).size().reset_index()

In [ ]:
df_size[df_size[0]>2].sort_values('sorted_nodes').shape

In [350]:
df_merged = df_place.merge(df_size[['sorted_nodes','length', 0]], on = ['sorted_nodes', 'length'])

In [359]:
def custom_agg(series):
    """
    Custom aggregation function:
    - If the number of elements is 4, compute the sum and divide by 2.
    - Otherwise, compute the mean.
    """
    n = len(series)
    if n == 4:
        return series.sum() / 2
    else:
        return series.mean()

In [382]:
l = clipped_df.groupby(['link_type_name', 'sorted_nodes', 'length'])['length'].agg(custom_agg).groupby('link_type_name').sum()

In [ ]:
1714983 * (1-.16), 1714983 * (1-.64), 1714983*165573 *.9, 30/5, 8*8, 2*8, 

In [ ]:
clipped_df.groupby(['link_type_name', 'sorted_nodes', 'length'])['length'].agg(custom_agg).groupby('link_type_name').sum().sum()

In [ ]:
l = clipped_df.groupby(['link_type_name', 'sorted_nodes', 'length']).agg({'length':'mean'}).groupby('link_type_name').sum()
l.squeeze(axis=1)

In [ ]:
16420549.68 -16421134.95

In [ ]:
X = clipped_df.groupby(['link_type_name', 'sorted_nodes', 'length']).size().reset_index()
X[X[0]>2]['length'].sum()

In [ ]:
clipped_df[clipped_df['sorted_nodes'] ==(26764, 256527)]

In [ ]:
        length  mean_length  
0  1931159.44   1931159.44  
1   873385.38    873385.38  
2  7649763.46   7649763.46  
3  4041302.82   4041302.82  
4  1668634.75   1668634.75  
5    91903.59     91903.59  
6   401122.36    401122.36 

In [ ]:
import dask_geopandas as dgpd
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon
import os

def clip_dask_geodataframe(dask_gdf, clip_geom):
    """
    Clip a Dask GeoDataFrame using a clipping geometry.
    
    Parameters:
    - dask_gdf: The Dask GeoDataFrame to be clipped.
    - clip_geom: The clipping geometry (should be a shapely geometry).

    Returns:
    - A Dask GeoDataFrame that has been clipped.
    """
    # Define a function to clip a single partition
    def clip_partition(partition, clip_geom):
        # Ensure the partition is a GeoDataFrame
        if not isinstance(partition, gpd.GeoDataFrame):
            partition = gpd.GeoDataFrame(partition, geometry='geometry')
        return partition.clip(mask=clip_geom)

    # Apply the clipping function to each partition
    clipped_dask_gdf = dask_gdf.map_partitions(clip_partition, clip_geom, meta=dask_gdf)
    
    return clipped_dask_gdf

def main():
    # Define the path to your polyline and multipolygon files
    polyline_path = 'path/to/your/polyline_data.shp'
    multipolygon_path = 'path/to/your/multipolygon_data.shp'
    
    # Load polyline and multipolygon data into Dask GeoDataFrames
    polyline_dask_gdf = dgpd.read_file(polyline_path, npartitions=4)
    multipolygon_gdf = gpd.read_file(multipolygon_path)
    
    # Ensure the output directory exists
    output_dir = 'path/to/save/clipped_data/'
    os.makedirs(output_dir, exist_ok=True)
    
    # Loop through each multipolygon and clip the polyline data
    for idx, row in multipolygon_gdf.iterrows():
        clip_geom = row['geometry']
        
        # Clip the polyline data using the current multipolygon
        clipped_dask_gdf = clip_dask_geodataframe(polyline_dask_gdf, clip_geom)
        
        # Compute the clipped data and save as a separate file
        clipped_df = clipped_dask_gdf.compute()
        clipped_df.to_file(os.path.join(output_dir, f'clipped_{idx}.shp'))

if __name__ == "__main__":
    main()


In [89]:
### above this line check ends

In [ ]:
# clipping roads at state level to places 
link_df_major = link_df[(link_df['link_type'] < 7) | (link_df['link_type'] == 15)]
state_streets_major = csv_2_gdf(link_df_major) 

# Create a new column with sorted tuples
state_streets_major['sorted_nodes'] = list(zip(state_streets_major['from_node_id'], state_streets_major['to_node_id']))
state_streets_major['sorted_nodes'] = state_streets_major['sorted_nodes'].apply(lambda x: tuple(sorted(x)))
   
clipped_to_places = []

state_places = gdf

# cretae a list of names and geoids
geoids = state_places.GEOID
name_of_places = state_places.NAMELSAD
gdf_wkt = state_places.geometry.to_wkt()  

# Clip link gdf by multipolygons
# Creates a list of clipped files per place in states_places
clipped_streets = [gpd.clip(state_streets_major, get_utm_projected(geom_wkt).to_crs(state_streets_major.crs)) for geom_wkt in gdf_wkt]

print(len(clipped_streets))
print(type(clipped_streets))


In [ ]:
clipped_projected_lengths = []
no_roads = []
name_no_roads = []

for i, clipped in enumerate(clipped_streets):
    clipped = clipped[~clipped.is_empty]
    # print(len(clipped))    
    print(f'Shape of the roadway clipped by place at index {i} is {clipped.shape}')

    if clipped.shape[0] != 0:
        output_df = pd.DataFrame(clipped)
        utm_length = []
        crs_length = []

        # convert geodataframe geometry to wkt format
        clipped_wkt = clipped.geometry.to_wkt()
        for geom in clipped_wkt:
            # # Convert to accurate UTM: https://gis.stackexchange.com/questions/436938/calculate-length-using-geopandas
            # print(geom)
            # read geometry as a shapely geometry
            line_shape = shapely.wkt.loads(geom)
            # indentify the centroid of the geometry
            utm_crs = utm_crs_from_latlon(lat = line_shape.centroid.y,
                                          lon = line_shape.centroid.x)       

            line_geo = gpd.GeoSeries([line_shape], crs='EPSG:4326')
            crs_length.append(line_geo.to_crs('EPSG:3857').length.values)
            utm_length.append(line_geo.to_crs(utm_crs).length.values)

        if output_df.shape[0] == len(utm_length):
            try:
                output_df['utm_length'] = utm_length
                output_df['crs_length'] = crs_length
            except Exception as e:        
                print(f"!!!====== ERROR ======!!! :\n  {file}")
                print(f"!!!File shapes do not match!!\n")
                continue

        clipped_projected_lengths.append(output_df)
    else:
        print('bleh')
        print('Places and geoids with missing geometry, so no x y values:====')
        print(geoids[i], name_of_places[i])
        no_roads.append(geoids[i])
        name_no_roads.append(name_of_places[i])
        # remove element at index i from the geoid list
        geoids.pop(i)
        name_of_places.pop(i)
        

In [ ]:
print(len(clipped_projected_lengths))

error_opening = []

motorway_roads = []
trunk_roads = []
primary_roads = []
secondary_roads = []
tertiary_roads = []
unclassified_roads = []
residential_roads = []

cl_motorway_roads = []
cl_trunk_roads = []
cl_primary_roads = []
cl_secondary_roads = []
cl_tertiary_roads = []
cl_unclassified_roads = []
cl_residential_roads = []

for k, clipped_links in enumerate(clipped_projected_lengths):
    try:                
        # Filter the data and calculate the sum of road lengths for each road type 
        clipped_links['utm_length'] = clipped_links['utm_length'].astype(str).str.replace("[","").str.replace("]","").astype(float)
        clipped_links['crs_length'] = clipped_links['crs_length'].astype(str).str.replace("[","").str.replace("]","").astype(float)
                    
        road_lengths = clipped_links.groupby('link_type_name')['utm_length'].sum()
        cl_road_lengths = clipped_links.groupby(['link_type_name', 'sorted_nodes', 'utm_length']).agg({'utm_length':'mean'}).groupby('link_type_name').sum()
        # print(cl_road_lengths)
        cl_road_lengths = cl_road_lengths.squeeze(axis=1)      
        
        # Append the calculated road lengths to the respective lists
        motorway_roads.append(road_lengths.get('motorway', 0))
        trunk_roads.append(road_lengths.get('trunk', 0))
        primary_roads.append(road_lengths.get('primary', 0))
        secondary_roads.append(road_lengths.get('secondary', 0))
        tertiary_roads.append(road_lengths.get('tertiary', 0))
        unclassified_roads.append(road_lengths.get('unclassified', 0))
        residential_roads.append(road_lengths.get('residential', 0))

        cl_motorway_roads.append(cl_road_lengths.get('motorway', 0))
        cl_trunk_roads.append(cl_road_lengths.get('trunk', 0))
        cl_primary_roads.append(cl_road_lengths.get('primary', 0))
        cl_secondary_roads.append(cl_road_lengths.get('secondary', 0))
        cl_tertiary_roads.append(cl_road_lengths.get('tertiary', 0))
        cl_unclassified_roads.append(cl_road_lengths.get('unclassified', 0))
        cl_residential_roads.append(cl_road_lengths.get('residential', 0))
        print("finished upto ", str(k))
    
            
    except Exception as e:
        error_opening.append(clipped_links.index)
        print(f"Error opening file clipped_links at position {k} of clipped_projected_lengths: {e}")

In [ ]:
list_of_tuples = list(zip(geoids, name_of_places, motorway_roads, trunk_roads, primary_roads, secondary_roads, 
                        tertiary_roads, unclassified_roads, residential_roads, cl_motorway_roads, cl_trunk_roads, cl_primary_roads,
                        cl_secondary_roads, cl_tertiary_roads, cl_unclassified_roads, cl_residential_roads))

df_roads = pd.DataFrame(list_of_tuples, columns = ['GEOID', 'NAMELSAD','motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential',
                                                'cl_motorway', 'cl_trunk', 'cl_primary', 'cl_secondary', 'cl_tertiary', 'cl_unclassified', 'cl_residential'])

print(df_roads.shape)

print(df_roads.shape)
# df_roads.to_csv(r'D:\Work\Box Sync\Quantify Infrastructure\OSM_raodway _length\All states\df_roads_utm_texas.csv')

missing_places = {no_roads[i]: name_no_roads[i] for i in range(len(no_roads))}
print(len(missing_places))
missing_places

In [ ]:
df_roads[['motorway', 'trunk', 'primary', 'secondary',
       'tertiary', 'unclassified', 'residential']].sum()*100/25336800.38648927

In [ ]:
df_roads[['cl_motorway', 'cl_trunk','cl_primary', 'cl_secondary', 'cl_tertiary', 'cl_unclassified',
       'cl_residential']].sum().sum()/1000

In [ ]:
df_place.groupby(['link_type','link_type_name', 'sorted_nodes', 'length']).agg({'length':'mean'}).groupby(['link_type','link_type_name']).sum()/1000